In [81]:
import pandas as pd
import re
import requests
from dateutil.relativedelta import relativedelta
from datetime import datetime
from genderize import Genderize

data_raw = pd.read_csv("data_raw/participantes.csv", sep = ";").drop(axis=1, labels=["Unnamed: 0"])

Unindo colunas de nascimento e renomeando

In [82]:
cols = ['Data de nascimento', 'Idade', 'Data de Nascimento']
data = data_raw.assign(
    data_nascimento = lambda x: x['Data de nascimento'].fillna("") +
                                x['Data de Nascimento'].fillna("") +
                                x['Idade'].fillna("")
).drop(cols, axis = 1).drop("Ref.", axis = 1)
data = data.rename(columns = {"Participante": 'nome', "Ocupação": "ocupacao", "Resultado":"resultado"})

In [83]:
def remover_nota(data):
    
    try:
        data = str(data)
        data = re.sub("\[(.*)\]", "", data)
        data = re.sub("\?", "", data)
        data = datetime.strptime(data, "%d/%m/%Y")
        return data
    except:
        return None    

data = data.assign(
    data_nascimento = data.apply(lambda x: remover_nota(x['data_nascimento']), axis = 1)
)

Corrigindo localidade de origem

In [84]:
data_local = data.assign(
    municipio = lambda x: x["Origem"].str.split(",").str[0],
    estado = lambda x: x["Origem"].str.split(",").str[1]
).drop(["Origem"], axis = 1)

data_local['estado'] = data_local['estado'].fillna("Exterior")

Corrigindo Resultado

In [85]:
num_participantes = data_local.groupby(["edicao"]).agg(
    n_participantes = ("edicao", "count")
)

participantes_edicao = data_local.join(num_participantes, on ='edicao').assign(
    resultado = lambda x: x['resultado'].str.split("em").str[0]
)

In [86]:
def arrumar_classificacao(row):
    # texto, n_participantes
    try:
        if any(i in row['resultado'] for i in ["Vencedor", "Vencedora"]):
            row['resultado'] = re.sub("Ven.*", "1", row['resultado'])
        elif any(i in row['resultado'] for i in ["eliminado", "eliminada"]):
            lugar = re.sub("ª eli.*|º eli.*", "", row['resultado'])
            lugar = row['n_participantes'] + 1 - int(lugar)
            row['resultado'] = str(lugar)
        elif "lugar" in row['resultado']: 
            row['resultado'] = row['resultado'][0]
        else:
            row['resultado'] = None 
    except:
        row['resultado'] = None
    
    return row['resultado']

participantes_edicao['classificacao'] = participantes_edicao.apply(arrumar_classificacao, axis = 1)
participantes_edicao = participantes_edicao.drop(['resultado'], axis = 1)

- Gênero #Não rodar esse codigo

In [27]:
def pegar_genero(lista):
    nome_simplificado = []
    nome_genero = {}
    for i in lista:
        nome = i.split()[0]
        nome_simplificado.append(nome)
    genero = Genderize().get(nome_simplificado)
    for n, i in enumerate(genero):
        nome_genero[lista[n]] = i
    return nome_genero

primeiro_nome = participantes_edicao['nome'].tolist()
generos_json = pegar_genero(primeiro_nome)
# participantes_edicao['genero'] = participantes_edicao.apply(genero, axis = 1)

In [79]:
participantes_genero = pd.DataFrame.from_dict(generos_json).transpose().reset_index().rename(columns = {"index": "nome"})
participantes_genero = participantes_genero[['nome', "gender"]]
participantes_genero.to_csv("data_raw/participantes_genero.csv", sep = ";", index = False)

In [93]:
participantes_completo = participantes_edicao.merge(participantes_genero, on = "nome").rename({"gender": "genero"}, axis = 1)
participantes_completo.to_csv("data/participantes_completo.csv", sep = ";", index = False)

- Idade quando o programa começou

In [95]:
from wikitable import wikitable

url = "https://pt.wikipedia.org/wiki/Big_Brother_Brasil"
temporadas_raw = wikitable(url)[2]

In [96]:
temporadas_raw.columns = temporadas_raw.columns.droplevel()

In [102]:
temporadas_raw

,Temporada,Temporada.1,Episódios,Estreia da temporada,Término da temporada
0,NaN,Big Brother Brasil 1,64,29 de janeiro de 2002,2 de abril de 2002
1,NaN,Big Brother Brasil 2,71,14 de maio de 2002,23 de julho de 2002
2,NaN,Big Brother Brasil 3,78,14 de janeiro de 2003,1 de abril de 2003
3,NaN,Big Brother Brasil 4,85,13 de janeiro de 2004,6 de abril de 2004
4,NaN,Big Brother Brasil 5,79,10 de janeiro de 2005,29 de março de 2005
5,NaN,Big Brother Brasil 6,78,10 de janeiro de 2006,28 de março de 2006
6,NaN,Big Brother Brasil 7,85,9 de janeiro de 2007,3 de abril de 2007
7,NaN,Big Brother Brasil 8,78,8 de janeiro de 2008,25 de março de 2008
8,NaN,Big Brother Brasil 9,85,13 de janeiro de 2009,7 de abril de 2009
9,NaN,Big Brother Brasil 10,78,12 de janeiro de 2010,30 de março de 2010


In [137]:
def calcular_idade(row):
    try:
        row["idade"] = relativedelta(row['estreia'], row['data_nascimento']).years
    except:
        row['idade'] = None
    return row['idade']

In [162]:
estreias = temporadas_raw[["Temporada.1", "Estreia da temporada"]].rename({"Temporada.1": "edicao", "Estreia da temporada": "estreia"}, axis = 1)

estreias['edicao'] = estreias['edicao'].str.split(" ").str[-1]
estreias['estreia'] = estreias['estreia'].str.replace(" de ", "/").str.replace("janeiro", "01").str.replace("maio", "05")

estreias['edicao'] = pd.to_numeric(estreias['edicao'])
participantes_estreia = participantes_completo.merge(estreias, on="edicao", how='left')

participantes_estreia['data_nascimento'] = participantes_estreia['data_nascimento'].apply(pd.to_datetime, format = "%Y/%m/%d")

participantes_estreia['estreia'] = participantes_estreia['estreia'].apply(pd.to_datetime, format = "%d/%m/%Y")

# participantes_estreia = participantes_estreia.assign(
#     idade_participacao = lambda x: x['estreia'].dt.year - x['data_nascimento'].dt.year
# ).drop(labels=['estreia', 'n_participantes'], axis = 1)

participantes_estreia['idade_participacao'] = participantes_estreia.apply(calcular_idade, axis = 1)

In [164]:
participantes_estreia = participantes_estreia[['nome', 'genero', "data_nascimento", "idade_participacao", "ocupacao", "municipio", "estado", "edicao", "classificacao"]]

In [165]:
participantes_estreia.to_csv("data/dados.csv", sep =';', index = False)